In [210]:
import urllib.request
import json
import requests
from bs4 import BeautifulSoup
import datetime
import time
import pandas as pd
import xlrd as xl
import numpy
import os

In [211]:
ano_inicio = 2004
#ano_inicio = 2021
now = datetime.datetime.now()
ano_actual = now.year

main_url = "https://wwwssl.aena.es/"

In [212]:
url_AENA_IATA_aeropuertos = "https://raw.githubusercontent.com/abediaga/AENA/main/AENA_IATA_aeropuertos.json"
file_name = "AENA_IATA_aeropuertos.json"

urllib.request.urlretrieve(url_AENA_IATA_aeropuertos, file_name)

aeropuertos_ = {}

with open('AENA_IATA_aeropuertos.json') as f:
    aeropuertos = json.load(f)


for aero_key, aero_value in aeropuertos.items():
    #aeropuertos_[aero_key] = aero_value["Nombre"]
    aeropuertos_[aero_value["Nombre"]] = aero_key
    if "Otros" in aero_value.keys():
        #print(aero_value["Otros"])
        for item in aero_value["Otros"]:
            #print(item)
            #aeropuertos_[aero_key] = str(item)
            aeropuertos_[str(item)] = aero_key


In [213]:
links_excels_AENA = {}

for ano in range(ano_inicio,ano_actual+1):
    url = "https://wwwssl.aena.es/csee/Satellite?SiteName=Estadisticas&anyo=" + str(ano) + "&c=Page&cid=1144247795704&pagename=Estadisticas%2FEstadisticas&periodoInforme=Mensual"
    response = requests.get(url)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    contenido = soup.find_all("td", {"class": "link2"})
    
    if len(contenido) > 0:
        for elemento in contenido:
            el = elemento.findAll("a", {"class": "link2"})
            if ".xls" in el[0]['href']:
                mes_ano = el[0].text.strip().partition(" ")
                mes = mes_ano[0]
                ano = int(mes_ano[-1])
                link = main_url + el[0]['href']
                if ano in links_excels_AENA:
                    links_excels_AENA[ano][mes] = link    
                else :
                    links_excels_AENA[ano] = {}
                    links_excels_AENA[ano][mes] = link

In [214]:
index = []
for ano in links_excels_AENA:
    for mes in links_excels_AENA[ano]:
        tupla = (ano, mes)
        index.append(tupla)

df_index = pd.MultiIndex.from_tuples(index, names=('Año', 'Mes'))

df_AENA_pasajeros = pd.DataFrame(columns=[], index=df_index)
df_AENA_operaciones = pd.DataFrame(columns=[], index=df_index)
df_AENA_mercancias = pd.DataFrame(columns=[], index=df_index)

In [215]:
for ano in links_excels_AENA:
    for mes in links_excels_AENA[ano]:

        file = requests.get(links_excels_AENA[ano][mes])
        filename = 'aena_temp.xls'
        open(filename, 'wb').write(file.content)
        
        Workbook = xl.open_workbook(filename)
        sheets = Workbook.sheet_names()
        
        hoja_seleccionada = ""
        for sheet in sheets:
            if ("TRÁFICO" in sheet) or ("TRAFICO" in sheet):
                hoja_seleccionada = sheet
        
        if ano < 2009:
            df = pd.read_excel(filename, sheet_name=hoja_seleccionada, skiprows=[0,1,2,3,4,5], usecols=[1,2,5,6,9,10])
        else:
            if ano == 2009:
                #print(hoja_seleccionada)
                df = pd.read_excel(filename, sheet_name=hoja_seleccionada, skiprows=[1,2,3,4,5,6,7,8], usecols=[2,4,7,8,11,12])
            else:
                if (ano == 2021) and (mes == "Marzo"):
                    df = pd.read_excel(filename, skiprows=[1,2,3,4,5,6,7,8], usecols=[2,3,7,8,12,13])
                    #display(df)
                else :
                    df = pd.read_excel(filename, skiprows=[1,2,3,4,5,6,7,8], usecols=[2,4,7,8,11,12])

        
        df_pasajeros = df[df.columns[0:2]].copy()
        df_pasajeros = df_pasajeros.set_index(df_pasajeros.columns[0])
        
        df_operaciones = df[df.columns[2:4]].copy()
        df_operaciones = df_operaciones.set_index(df_operaciones.columns[0])
        
        df_mercancias = df[df.columns[4:6]].copy()
        df_mercancias = df_mercancias.set_index(df_mercancias.columns[0])

        df_pasajeros.index.name = 'Aeropuerto'
        df_operaciones.index.name = 'Aeropuerto'
        df_mercancias.index.name = 'Aeropuerto'

        df_pasajeros = df_pasajeros[df_pasajeros.index.notnull()]
        df_operaciones = df_operaciones[df_operaciones.index.notnull()]
        df_mercancias = df_mercancias[df_mercancias.index.notnull()]
        
        for index, row in df_pasajeros.iterrows():
            index = index.strip()
            if (index in aeropuertos_.keys()):
                if type(row.iloc[0]) == numpy.float64:
                    valor = row.iloc[0].astype(int)
                if type(row.iloc[0]) == float:
                    valor = int(row.iloc[0].astype(int))
                if type(row.iloc[0]) == str:
                    valor = 0
                
                df_AENA_pasajeros.loc[(ano,mes),aeropuertos_[index]] = row.iloc[0]
            #else :
                #print("El código: <" + str(index) + "> no está entre los códigos IATA del fichero JSON.")

        for index, row in df_operaciones.iterrows():
            index = index.strip()
            if (index in aeropuertos_.keys()):
                if type(row.iloc[0]) == numpy.float64:
                    valor = row.iloc[0].astype(int)
                if type(row.iloc[0]) == float:
                    valor = int(row.iloc[0].astype(int))
                if type(row.iloc[0]) == str:
                    valor = 0
                df_AENA_operaciones.loc[(ano,mes),aeropuertos_[index]] = row.iloc[0]
            #else :
                #print("El código: <" + str(index) + "> no está entre los códigos IATA del fichero JSON.")
        
        for index, row in df_mercancias.iterrows():
            index = index.strip()
            if (index in aeropuertos_.keys()):
                if type(row.iloc[0]) == numpy.float64:
                    valor = row.iloc[0].astype(int)
                if type(row.iloc[0]) == float:
                    valor = int(row.iloc[0].astype(int))
                if type(row.iloc[0]) == str:
                    valor = 0
                df_AENA_mercancias.loc[(ano,mes),aeropuertos_[index]] = row.iloc[0]
            #else :
                #print("El código: <" + str(index) + "> no está entre los códigos IATA del fichero JSON.")



In [216]:
#DELETE 'aena_temp.xls'
os.remove("aena_temp.xls")

In [217]:
#display(df_AENA_operaciones.loc[[2020,2021]])
#display(df_AENA_operaciones.index)
#display(df_AENA_pasajeros.loc[(2019,"Enero")])

In [218]:
#Exportamos el dataframe de Mercabilbao a un Pickle
pd.to_pickle(df_AENA_pasajeros, "./raw_df_AENA_pasajeros.pkl")
pd.to_pickle(df_AENA_operaciones, "./raw_df_AENA_operaciones.pkl")
pd.to_pickle(df_AENA_mercancias, "./raw_df_AENA_mercancias.pkl")